In [62]:
# Import dependencies
import pandas as pd
import numpy as np
import json

from config import GEOAPIFY_API
import requests
import pprint
import time

pd.set_option('max_colwidth', 400)

In [16]:
# Read the data into a Pandas DataFrame
shark_data_raw_df = pd.read_excel("Resources/GSAF5-1.xlsx")
shark_data_raw_df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2019.08.27,27-Aug-2019,2019.0,Provoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Donald Walsh,M,...,6.5' shark,"K. McMurray, TrackingSharks.com",2019.08.27-Walsh.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.27-Walsh.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.27-Walsh.pdf,2019.08.27,2019.08.27,6452.0,NaN,NaN
1,2019.08.21,21-Aug-2019,2019.0,Unprovoked,USA,Florida,"Fort Lauderdale, Broward County",Wading,Christian Mariani,M,...,NaN,"K. McMurray, TrackingSharks.com",2019.08.21-Mariani.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.21-Mariani.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.21-Mariani.pdf,2019.08.21,2019.08.21,6451.0,NaN,NaN
2,2019.08.20,20-Aug-2019,2019.0,Unprovoked,USA,Hawaii,"Napoopoo Park in Kealakekua Bay, The Big Island",Swimming,female,F,...,NaN,"Hawaii News Now, 8/20/2019",2019.08.20-Hawaii.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.20-Hawaii.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.20-Hawaii.pdf,2019.08.20,2019.08.20,6450.0,NaN,NaN
3,2019.08.16,16-Aug-2019,2019.0,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Wading,Maggie Crum,M,...,NaN,"WESH, 8/16/2019",2019.08.16-Crum.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.16-Crum.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.16-Crum.pdf,2019.08.16,2019.08.16,6449.0,NaN,NaN
4,2019.08.11,11-Aug-2019,2019.0,Unprovoked,USA,Hawaii,"Makaha Beach, Oahu",Surfing,Max Keliikipi,M,...,10' to 12' shark,"M. Michaelson, GSAF",2019.08.11-Keliikipi.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.11-Keliikipi.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.11-Keliikipi.pdf,2019.08.11,2019.08.11,6448.0,NaN,NaN


In [17]:
# Get a brief summary of the shark_data_raw_df DataFrame
shark_data_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25760 entries, 0 to 25759
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8739 non-null   object 
 1   Date                    6451 non-null   object 
 2   Year                    6449 non-null   float64
 3   Type                    6447 non-null   object 
 4   Country                 6401 non-null   object 
 5   Area                    5988 non-null   object 
 6   Location                5904 non-null   object 
 7   Activity                5901 non-null   object 
 8   Name                    6241 non-null   object 
 9   Sex                     5886 non-null   object 
 10  Age                     3586 non-null   object 
 11  Injury                  6423 non-null   object 
 12  Fatal (Y/N)             5905 non-null   object 
 13  Time                    3064 non-null   object 
 14  Species                 3545 non-null 

In [54]:
# Limit the attacks to those that occurred in the United States
shark_data_usa_df = shark_data_raw_df[shark_data_raw_df["Country"] == "USA"]
shark_data_usa_df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2019.08.27,27-Aug-2019,2019.0,Provoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Donald Walsh,M,...,6.5' shark,"K. McMurray, TrackingSharks.com",2019.08.27-Walsh.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.27-Walsh.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.27-Walsh.pdf,2019.08.27,2019.08.27,6452.0,NaN,NaN
1,2019.08.21,21-Aug-2019,2019.0,Unprovoked,USA,Florida,"Fort Lauderdale, Broward County",Wading,Christian Mariani,M,...,NaN,"K. McMurray, TrackingSharks.com",2019.08.21-Mariani.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.21-Mariani.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.21-Mariani.pdf,2019.08.21,2019.08.21,6451.0,NaN,NaN
2,2019.08.20,20-Aug-2019,2019.0,Unprovoked,USA,Hawaii,"Napoopoo Park in Kealakekua Bay, The Big Island",Swimming,female,F,...,NaN,"Hawaii News Now, 8/20/2019",2019.08.20-Hawaii.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.20-Hawaii.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.20-Hawaii.pdf,2019.08.20,2019.08.20,6450.0,NaN,NaN
3,2019.08.16,16-Aug-2019,2019.0,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Wading,Maggie Crum,M,...,NaN,"WESH, 8/16/2019",2019.08.16-Crum.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.16-Crum.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.16-Crum.pdf,2019.08.16,2019.08.16,6449.0,NaN,NaN
4,2019.08.11,11-Aug-2019,2019.0,Unprovoked,USA,Hawaii,"Makaha Beach, Oahu",Surfing,Max Keliikipi,M,...,10' to 12' shark,"M. Michaelson, GSAF",2019.08.11-Keliikipi.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.11-Keliikipi.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.11-Keliikipi.pdf,2019.08.11,2019.08.11,6448.0,NaN,NaN


In [55]:
# Get a brief summary of the shark_data_usa_df DataFrame
shark_data_usa_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2310 entries, 0 to 6446
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             2309 non-null   object 
 1   Date                    2310 non-null   object 
 2   Year                    2310 non-null   float64
 3   Type                    2310 non-null   object 
 4   Country                 2310 non-null   object 
 5   Area                    2305 non-null   object 
 6   Location                2256 non-null   object 
 7   Activity                2173 non-null   object 
 8   Name                    2279 non-null   object 
 9   Sex                     2211 non-null   object 
 10  Age                     1563 non-null   object 
 11  Injury                  2305 non-null   object 
 12  Fatal (Y/N)             2119 non-null   object 
 13  Time                    1450 non-null   object 
 14  Species                 1366 non-null   objec

In [56]:
# Remove rows where Location is NaN 
shark_data_usa_df = shark_data_usa_df.dropna(subset=["Location"])
shark_data_usa_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2256 entries, 0 to 6446
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             2255 non-null   object 
 1   Date                    2256 non-null   object 
 2   Year                    2256 non-null   float64
 3   Type                    2256 non-null   object 
 4   Country                 2256 non-null   object 
 5   Area                    2255 non-null   object 
 6   Location                2256 non-null   object 
 7   Activity                2126 non-null   object 
 8   Name                    2225 non-null   object 
 9   Sex                     2161 non-null   object 
 10  Age                     1544 non-null   object 
 11  Injury                  2251 non-null   object 
 12  Fatal (Y/N)             2072 non-null   object 
 13  Time                    1438 non-null   object 
 14  Species                 1345 non-null   objec

In [57]:
# Change headers to lowercase and replace spaces with underscores
shark_data_usa_df.columns = shark_data_usa_df.columns.str.lower().str.replace(" ", "_")
shark_data_usa_df.head()

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2019.08.27,27-Aug-2019,2019.0,Provoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Donald Walsh,M,...,6.5' shark,"K. McMurray, TrackingSharks.com",2019.08.27-Walsh.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.27-Walsh.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.27-Walsh.pdf,2019.08.27,2019.08.27,6452.0,NaN,NaN
1,2019.08.21,21-Aug-2019,2019.0,Unprovoked,USA,Florida,"Fort Lauderdale, Broward County",Wading,Christian Mariani,M,...,NaN,"K. McMurray, TrackingSharks.com",2019.08.21-Mariani.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.21-Mariani.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.21-Mariani.pdf,2019.08.21,2019.08.21,6451.0,NaN,NaN
2,2019.08.20,20-Aug-2019,2019.0,Unprovoked,USA,Hawaii,"Napoopoo Park in Kealakekua Bay, The Big Island",Swimming,female,F,...,NaN,"Hawaii News Now, 8/20/2019",2019.08.20-Hawaii.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.20-Hawaii.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.20-Hawaii.pdf,2019.08.20,2019.08.20,6450.0,NaN,NaN
3,2019.08.16,16-Aug-2019,2019.0,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Wading,Maggie Crum,M,...,NaN,"WESH, 8/16/2019",2019.08.16-Crum.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.16-Crum.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.16-Crum.pdf,2019.08.16,2019.08.16,6449.0,NaN,NaN
4,2019.08.11,11-Aug-2019,2019.0,Unprovoked,USA,Hawaii,"Makaha Beach, Oahu",Surfing,Max Keliikipi,M,...,10' to 12' shark,"M. Michaelson, GSAF",2019.08.11-Keliikipi.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.11-Keliikipi.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.11-Keliikipi.pdf,2019.08.11,2019.08.11,6448.0,NaN,NaN


In [58]:
# Get the first 5 rows of the shark_data_usa_df DataFrame
shark_short_data_usa_df = shark_data_usa_df[:5]
shark_short_data_usa_df.head(10)

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2019.08.27,27-Aug-2019,2019.0,Provoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Donald Walsh,M,...,6.5' shark,"K. McMurray, TrackingSharks.com",2019.08.27-Walsh.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.27-Walsh.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.27-Walsh.pdf,2019.08.27,2019.08.27,6452.0,NaN,NaN
1,2019.08.21,21-Aug-2019,2019.0,Unprovoked,USA,Florida,"Fort Lauderdale, Broward County",Wading,Christian Mariani,M,...,NaN,"K. McMurray, TrackingSharks.com",2019.08.21-Mariani.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.21-Mariani.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.21-Mariani.pdf,2019.08.21,2019.08.21,6451.0,NaN,NaN
2,2019.08.20,20-Aug-2019,2019.0,Unprovoked,USA,Hawaii,"Napoopoo Park in Kealakekua Bay, The Big Island",Swimming,female,F,...,NaN,"Hawaii News Now, 8/20/2019",2019.08.20-Hawaii.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.20-Hawaii.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.20-Hawaii.pdf,2019.08.20,2019.08.20,6450.0,NaN,NaN
3,2019.08.16,16-Aug-2019,2019.0,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Wading,Maggie Crum,M,...,NaN,"WESH, 8/16/2019",2019.08.16-Crum.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.16-Crum.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.16-Crum.pdf,2019.08.16,2019.08.16,6449.0,NaN,NaN
4,2019.08.11,11-Aug-2019,2019.0,Unprovoked,USA,Hawaii,"Makaha Beach, Oahu",Surfing,Max Keliikipi,M,...,10' to 12' shark,"M. Michaelson, GSAF",2019.08.11-Keliikipi.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.11-Keliikipi.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.11-Keliikipi.pdf,2019.08.11,2019.08.11,6448.0,NaN,NaN


In [61]:
# Add a new column to the DataFrame to store the coordinates
shark_short_data_usa_df["latitude"] = np.nan
shark_short_data_usa_df["longitude"] = np.nan

# Set parameters to get the latitude and longitude of the Location
params = {"apiKey": GEOAPIFY_API}

# Loop through the rows of the shark_short_data_usa_df DataFrame
for index, row in shark_short_data_usa_df.iterrows():
    # Get the location from the row
    params["text"] = row["location"]

    # Set the base URL
    base_url = "https://api.geoapify.com/v1/geocode/search?"

    geo_location = requests.get(base_url, params=params).json()

    # Get the coordinates from the response and add them to the DataFrame
    shark_short_data_usa_df.loc[index, "latitude"] = geo_location["features"][0]["properties"]["lat"]
    shark_short_data_usa_df.loc[index, "longitude"] = geo_location["features"][0]["properties"]["lon"]

# Get the first 5 rows of the shark_short_data_usa_df DataFrame
shark_short_data_usa_df.head()

C:\Users\carlo\AppData\Local\Temp\ipykernel_21536\2058876470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_short_data_usa_df["latitude"] = np.nan
C:\Users\carlo\AppData\Local\Temp\ipykernel_21536\2058876470.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_short_data_usa_df["longitude"] = np.nan


29.0258132
-80.9271271
26.1223084
-80.1433786
51.364983
-0.473212
29.0258132
-80.9271271
-12.206631
-77.014699


,case_number,date,year,type,country,area,location,activity,name,sex_,...,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23,latitude,longitude
0,2019.08.27,27-Aug-2019,2019.0,Provoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Donald Walsh,M,...,2019.08.27-Walsh.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.27-Walsh.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.27-Walsh.pdf,2019.08.27,2019.08.27,6452.0,NaN,NaN,29.025813,-80.927127
1,2019.08.21,21-Aug-2019,2019.0,Unprovoked,USA,Florida,"Fort Lauderdale, Broward County",Wading,Christian Mariani,M,...,2019.08.21-Mariani.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.21-Mariani.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.21-Mariani.pdf,2019.08.21,2019.08.21,6451.0,NaN,NaN,26.122308,-80.143379
2,2019.08.20,20-Aug-2019,2019.0,Unprovoked,USA,Hawaii,"Napoopoo Park in Kealakekua Bay, The Big Island",Swimming,female,F,...,2019.08.20-Hawaii.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.20-Hawaii.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.20-Hawaii.pdf,2019.08.20,2019.08.20,6450.0,NaN,NaN,51.364983,-0.473212
3,2019.08.16,16-Aug-2019,2019.0,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Wading,Maggie Crum,M,...,2019.08.16-Crum.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.16-Crum.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.16-Crum.pdf,2019.08.16,2019.08.16,6449.0,NaN,NaN,29.025813,-80.927127
4,2019.08.11,11-Aug-2019,2019.0,Unprovoked,USA,Hawaii,"Makaha Beach, Oahu",Surfing,Max Keliikipi,M,...,2019.08.11-Keliikipi.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.11-Keliikipi.pdf,http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2019.08.11-Keliikipi.pdf,2019.08.11,2019.08.11,6448.0,NaN,NaN,-12.206631,-77.014699


In [ ]:
# Add a new column to the DataFrame to store the coordinates
shark_data_usa_df["latitude"] = np.nan
shark_data_usa_df["longitude"] = np.nan

# Set parameters to get the latitude and longitude of the Location
params = {"apiKey": GEOAPIFY_API}

# Loop through the rows of the shark_short_data_usa_df DataFrame
for index, row in shark_data_usa_df.iterrows():
    # Get the location from the row
    params["text"] = row["location"]

    # Set the base URL
    base_url = "https://api.geoapify.com/v1/geocode/search?"

    geo_location = requests.get(base_url, params=params).json()

    # Get the coordinates from the response and add them to the DataFrame
    shark_data_usa_df.loc[index, "latitude"] = geo_location["features"][0]["properties"]["lat"]
    shark_data_usa_df.loc[index, "longitude"] = geo_location["features"][0]["properties"]["lon"]

    # Pause for 1 second to avoid hitting the rate limit
    time.sleep(1)

# Get the first 5 rows of the shark_short_data_usa_df DataFrame
shark_data_usa_df.head()